<a href="https://colab.research.google.com/github/ssgalitsky/PedalNetRT/blob/master/colab_TPU_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 To Train models, upload the pedalnet repository to your Google Drive account and run the following code. In Colab options, go to "Runtime", "Change Runtime Type", and change hardware accelerator to "TPU".
 
 Note: User needs to modify the train.py script to use TPUs. Uncomment line as instructed in the file notes.

 Note: Having difficulty saving and loading TPU model after converting to CPU. It can be done, but currently needs some work.

In [ ]:
!git clone https://github.com/ssgalitsky/PedalNetRT.git

!pip3 install torchvision
!pip install pytorch-lightning

!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version nightly --apt-packages libomp5 libopenblas-dev
!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.6-cp36-cp36m-linux_x86_64.whl

In [2]:
import torch
use_cuda=True
if use_cuda and torch.cuda.is_available():
  net.cuda()

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')


In [ ]:
import os
os.chdir('/content/PedalNetRT')
os.getcwd()


In [ ]:
#!ffmpeg -i "data/guitar.wav" -af "pan=stereo|c0=c0|c1=c0" "data/guitar.wav"
#!ffmpeg -i "data/flute.wav" -af "pan=stereo|c0=c0|c1=c0" "data/flute.wav"


In [7]:
!python3 "prepare_data.py" "/content/PedalNetRT/data/guitar.wav" "/content/PedalNetRT/data/flute.wav"


prepare_data.py:8: WavFileWarning: Chunk (non-data) not understood, skipping it.
  in_rate, in_data = wavfile.read(args.in_file)
prepare_data.py:9: WavFileWarning: Chunk (non-data) not understood, skipping it.
  out_rate, out_data = wavfile.read(args.out_file)


In [ ]:

import os
assert os.environ['COLAB_TPU_ADDR']
import pytorch_lightning
!python3 "train.py" --batch_size=32 --max_epochs=1500 --learning_rate=3e-3 --num_channels=5 

GPU available: False, used: False
TPU available: True, using: 8 TPU cores
training on 8 TPU cores
INIT TPU local core: 0, global rank: 0 with XLA_USE_BF16=None
INIT TPU local core: 1, global rank: 1 with XLA_USE_BF16=None
2020-10-12 00:48:54.013451: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
INIT TPU local core: 2, global rank: 2 with XLA_USE_BF16=None
INIT TPU local core: 6, global rank: 6 with XLA_USE_BF16=None
INIT TPU local core: 3, global rank: 3 with XLA_USE_BF16=None
INIT TPU local core: 7, global rank: 7 with XLA_USE_BF16=None
INIT TPU local core: 5, global rank: 5 with XLA_USE_BF16=None
INIT TPU local core: 4, global rank: 4 with XLA_USE_BF16=None

  | Name    | Type    | Params
------------------------------------
0 | wavenet | WaveNet | 1 K   
Validation sanity check: 100% 2/2 [00:02<00:00,  1.09s/it]/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: The

In [ ]:
import torch_xla
import torch_xla.core.xla_model as xm

from model import PedalNet
# Change path below to match model file
model = PedalNet.load_from_checkpoint('lightning_logs/version_0/checkpoints/epoch=1.ckpt')

# xm.save(model.state_dict(), 'tpu_to_cpu.ckpt') 
xm.save(model, 'tpu_to_cpu_model.ckpt') 
# issues loading model, see pytorch_lightning issues #2303 and #3044 (Might be completed by now)

# TODO: Add code to be able to load saved model sucessfully
